In [1]:
import pandas as pd 
import os
import plotly.express as px # version ()
import plotly.graph_objects as go

import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output


# importing and clean data ( importing csv into pandas)

In [ ]:

# Set your path to the folder containing the .csv files
PATH = '../data/Analysis/' # Use your path

# Fetch all files in path
fileNames = os.listdir(PATH)

# Filter file name list for files ending with .csv
fileNames = [file for file in fileNames if '.csv' in file]

# Loop over all files

my_list = []

for file in fileNames:
# Read .csv file and append to list
    df = pd.read_csv(PATH + file, index_col= None, header=0)
    df['Datetime']= pd.to_datetime(df['Datetime'])
    df['Name'] = file.replace('_results.csv','')
    my_list.append(df)  
        


In [ ]:
energy_con_dataframe = pd.concat(my_list, axis=0, ignore_index=True)
energy_con_dataframe = energy_con_dataframe.drop(columns=['obs_ci_lower','obs_ci_upper'],)
energy_con_dataframe['Datetime']= pd.to_datetime(energy_con_dataframe['Datetime'])
# energy_con_dataframe = energy_con_dataframe.groupby('Datetime')['Actual','Predicted','Name'].mean().round(1)
# energy_con_dataframe.to_csv('../data/energy_con_df.csv')
energy_con_dataframe.head()

KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.tslibs.tzconversion.tz_convert_single'
Traceback (most recent call last):
  File "pandas/_libs/tslibs/tzconversion.pyx", line 519, in pandas._libs.tslibs.tzconversion._tz_convert_dst
  File "pandas/_libs/tslibs/timezones.pyx", line 220, in pandas._libs.tslibs.timezones.get_dst_info
  File "pandas/_libs/tslibs/timezones.pyx", line 155, in pandas._libs.tslibs.timezones.get_utcoffset
  File "/Applications/anaconda3/lib/python3.8/site-packages/pytz/__init__.py", line 401, in utcoffset
    def utcoffset(self, dt):
KeyboardInterrupt: 


KeyboardInterrupt: 

Exception ignored in: 'pandas._libs.tslibs.tzconversion.tz_convert_single'
Traceback (most recent call last):
  File "pandas/_libs/tslibs/tzconversion.pyx", line 519, in pandas._libs.tslibs.tzconversion._tz_convert_dst
  File "pandas/_libs/tslibs/timezones.pyx", line 220, in pandas._libs.tslibs.timezones.get_dst_info
  File "pandas/_libs/tslibs/timezones.pyx", line 155, in pandas._libs.tslibs.timezones.get_utcoffset
  File "/Applications/anaconda3/lib/python3.8/site-packages/pytz/__init__.py", line 401, in utcoffset
    def utcoffset(self, dt):
KeyboardInterrupt: 


## Cleaning up data and creating lables for graphing

<!-- 
# UNCG_baseball_results = pd.read_excel('../data/Meter_Names_and_Labels.xslx')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_stadium_prk_light_results = pd.read_csv('../data/Analysis/BaseballStadiumPrkLights_results.csv')
# # UNCG_baseball_stadium_rec_filed_light_results = pd.read_csv('../data/Analysis/BaseballStadiumRecFieldLights_results.csv')
# fileName = ['Baseball',
# 'BryanBldg',
# 'CampusSupplyStore', 
# 'CarmichaelBldg',
# 'ChemicalSafety', 
# 'ConeArtBldg',
# 'CurryBldg',
# 'Eberhart',
# 'ElliottUnivCenter', 
# 'Graham',
# 'GrayHome', 
# 'GroganHall',
# 'GuilfordResHall',
# 'HHP',
# 'HHPSoccerField',
# 'JLibraryOldEastAtDoor', 
# 'JLibraryOldEastPanelP',
# 'JacksonLibChiller', 
# 'MaryFoustHall',
# 'McIverDeckChillerSvc3',
# 'McIverStPrkgDeck',
# 'MooreNursingBldg',
# 'MooreStrongResHall',
# 'MooreStrongSiteLts011',
# 'MooreStrongSiteLts127',
# 'Mossman',
# 'OaklandAveParkingDeck',
# 'PettyBldg',
# 'ReynoldsHall',
# 'SchOfMusic',
# 'SinkGroundsBldg',
# 'SinkOffices' ,
# 'Spencer' - "Spencer Hall",
# 'SpringGardenAptsCStore',
# 'SteamPlant',
# 'StoneBuilding',
# 'StudentRecCenterMain',
# 'SullivanScience',
# 'TaylorTheatre',
# 'TowerVillage',
# 'VisitorCenter',
# 'WalkerDeck',
# 'WeilWinfield']




# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_results = pd.read_csv('../data/Analysis/Baseball_results.csv')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_results = pd.read_csv('../data/Analysis/Baseball_results.csv')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_results = pd.read_csv('../data/Analysis/Baseball_results.csv')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_results = pd.read_csv('../data/Analysis/Baseball_results.csv')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_results = pd.read_csv('../data/Analysis/Baseball_results.csv')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_results = pd.read_csv('../data/Analysis/Baseball_results.csv')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_results = pd.read_csv('../data/Analysis/Baseball_results.csv')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_results = pd.read_csv('../data/Analysis/Baseball_results.csv')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_results = pd.read_csv('../data/Analysis/Baseball_results.csv')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
# # UNCG_baseball_results = pd.read_csv('../data/Analysis/Baseball_results.csv')
# # UNCG_baseball_field_support_bldg_results = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')

# # UNCG_baseball_results['building'] ='Baseball'
# # UNCG_baseball_field_support_bldg_results['building'] = 'Baseball Field Support Building'
# # UNCG_baseball_stadium_prk_light_results['building'] = 'Baseball Stadium Park Light'
# # # UNCG_baseball_stadium_rec_filed_light_results['building'] = 'Baseball Stadium Rec. Filed Light'


# # # df = pd.concat([UNCG_baseball_results,UNCG_baseball_field_support_bldg_results,UNCG_baseball_stadium_rec_filed_light_results,UNCG_baseball_stadium_prk_light_results])
# # UNCG_baseball_results['Datetime'] = pd.to_datetime(UNCG_baseball_results['Datetime'])

# # UNCG_baseball_results = UNCG_baseball_results.groupby(['building','Datetime'])[['Actual','Predicted','obs_ci_lower','obs_ci_upper']].mean()

# # UNCG_baseball_results.reset_index(inplace=True)
# df = pd.DataFrame(UNCG_baseball_results)
# df.head(80)
 -->>>>>

In [3]:
energy_con_df = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv')
energy_con_df['Datetime']= pd.to_datetime(energy_con_df['Datetime'])
energy_con_df.groupby('Datetime')['Actual','Predicted'].mean().round(1)

<ipython-input-3-3ff7d399d3ea>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  energy_con_df.groupby('Datetime')['Actual','Predicted'].mean().round(1)


,Actual,Predicted
Datetime,,
2015-06-01 00:00:00-04:00,1.0,1.3
2015-06-01 01:00:00-04:00,2.0,1.2
2015-06-01 02:00:00-04:00,1.0,1.2
2015-06-01 03:00:00-04:00,1.0,1.2
2015-06-01 04:00:00-04:00,1.0,1.3
...,...,...
2020-11-12 19:00:00-05:00,1.0,4.1
2020-11-12 20:00:00-05:00,1.0,4.5
2020-11-12 21:00:00-05:00,1.0,4.7


In [31]:
energy_con_df = pd.read_csv('../data/Analysis/BaseballFieldSupportBldg_results.csv',index_col='Datetime')
# energy_con_df = energy_con_df.drop(columns='Unnamed: 0')
# energy_con_df.index= pd.to_datetime(energy_con_df)
# GB=energy_con_df.groupby([(energy_con_df.index.year),(energy_con_df.index.month)]).sum()

# energy_con_df = energy_con_df[energy_con_df['Datetime','Actual','Predicted']]

# date_time_lable= pd.to_datetime(energy_con_df['Datetime'])
# daily = energy_con_df.dt.to_period("M")  # new way to get the same

# dg = energy_con_df.groupby(daily)[['Datetime','Actual','Predicted']].sum()
# GB.head()
energy_con_df

,Actual,Predicted,obs_ci_lower,obs_ci_upper
Datetime,,,,
2015-06-01 00:00:00-04:00,1.0,1.290190,-1.033956,3.614336
2015-06-01 01:00:00-04:00,2.0,1.243702,-1.080407,3.567811
2015-06-01 02:00:00-04:00,1.0,1.225347,-1.098767,3.549460
2015-06-01 03:00:00-04:00,1.0,1.221025,-1.103087,3.545136
2015-06-01 04:00:00-04:00,1.0,1.300620,-1.023491,3.624731
...,...,...,...,...
2020-11-12 19:00:00-05:00,1.0,4.108189,1.750434,6.465943
2020-11-12 20:00:00-05:00,1.0,4.485297,2.127834,6.842761
2020-11-12 21:00:00-05:00,1.0,4.654869,2.297384,7.012354


In [17]:

date_time_lable = pd.to_datetime(energy_con_df['Datetime']).dt.to_period("D")
date_time_lable.head()


IndexError: Column(s) ['Datetime', 'Actual', 'Predicted'] already selected

# App Layout

In [4]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1(children = 'UNCG Engergy Consumptions',
           style={
               'textAlign' : 'center',
               'color' : '#456FBV'
           }),
    html.Div(children='funded by Green Fund',
            style={
                'textAlign' : 'center',
                'color' : '#456FBV'
            }),
    
    dcc.Graph(
        id='overall-consumption-graph',
        figure={
            'data': [
                {'x': name_meter_loc_lable, 'y':actual_lable, 'type': 'bar', 'name': 'Actual'},
                {'x': name_meter_loc_lable, 'y': predicted_lable, 'type': 'bar', 'name': u'Predicted'},
            ],
            'layout': {
                'title': 'Dash Data Visualization'
            }
        }
    )
])

In [ ]:
mgr_options = energy_con_dataframe["Name"].unique()

app = dash.Dash()

app.layout = html.Div([
    html.H2("Sales Funnel Report"),
    html.Div(
        [
            dcc.Dropdown(
                id="Name",
                options=[{
                    'label': i,
                    'value': i
                } for i in mgr_options],
                value='All Managers'),
        ],
        style={'width': '25%',
               'display': 'inline-block'}),
    dcc.Graph(id='funnel-graph'),
])


@app.callback(
    dash.dependencies.Output('funnel-graph', 'figure'),
    [dash.dependencies.Input('Name', 'value')])
def update_graph(Manager):
    if Manager == "All Managers":
        df_plot = df.copy()
    else:
        df_plot = df[df['Name'] == Manager]

    pv = pd.pivot_table(
        df_plot,
        index=['Name'],
        columns=["Datetime"],
        values=['Actual'],
        aggfunc=sum,
        fill_value=0)

    trace1 = go.Bar(x=pv.index, y=pv[('Quantity', 'declined')], name='Declined')
    trace2 = go.Bar(x=pv.index, y=pv[('Quantity', 'pending')], name='Pending')
    trace3 = go.Bar(x=pv.index, y=pv[('Quantity', 'presented')], name='Presented')
    trace4 = go.Bar(x=pv.index, y=pv[('Quantity', 'won')], name='Won')

    return {
        'data': [trace1, trace2, trace3, trace4],
        'layout':
        go.Layout(
            title='Customer Order Status for {}'.format(Manager),
            barmode='stack')
    }


In [5]:
if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Nov/2020 19:12:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 19:12:44] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 19:12:50] "GET /_favicon.ico?v=1.17.0 HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 19:12:53] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 19:29:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 19:29:26] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2020 19:30:01] "GET /_dash-layout HTTP/1.1" 200 -
